In [1]:
import sys, os, subprocess
from datetime import date
from io import StringIO
import pandas as pd, numpy as np
import time, timeit, cython
from numba import jit, prange, njit

## Extracting Data

In [2]:
dist = pd.read_csv('dist.csv',sep=',',index_col=[0,1,2,3], header=[0,1,2,3])
pearson = pd.read_csv('pearson.csv', sep=',', index_col=[0,1,2,3], header=[0,1,2,3])

display(dist.head(5))
display(pearson.head(5))

,,,CompoundName,SW219778-1,SW219366-1,SW218151-2,SW219425-1,SW219079-1,SW222338-1,SW219396-1,SW196607-4,SW219879-1,SW219768-1,...,SW219416-1,SW218112-2,SW219754-1,SW220254-1,SW219869-1,SW219664-1,SW197554-3,SW219415-1,SW199536-4,SW218141-2
,,,Concentration,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
,,,PlateName,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,...,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142
,,,WellID,L08,O19,I04,K11,J06,B04,J05,G20,H11,M13,...,P20,C17,D20,K16,L20,I09,J16,J21,M09,E15
CompoundName,Concentration,PlateName,WellID,,,,,,,,,,,,,,,,,,,,,
SW218124-2,10.0,SP0135,E03,1.125537,0.983073,1.331452,0.904111,0.834197,0.933505,0.687167,1.377220,1.265864,0.746083,...,0.885127,1.234141,1.173954,1.227687,0.687337,0.555437,1.276599,1.069406,0.551949,1.072620
SW219896-1,10.0,SP0135,O15,1.247795,1.024736,1.511265,1.294668,0.754919,0.685771,0.671116,1.197581,1.184858,1.017520,...,0.937583,1.131026,1.254594,1.295335,0.974856,0.699858,1.324513,0.970824,0.731570,1.080931
SW197639-3,10.0,SP0135,N08,0.627213,0.731539,0.645619,1.153401,1.253436,1.080003,1.257563,0.733493,0.761019,0.867918,...,1.103231,0.695401,0.723387,1.120444,0.934785,1.300075,0.654995,0.895650,1.224923,0.634015
SW218086-2,10.0,SP0135,G22,1.356639,1.220774,1.513809,1.122234,0.799518,0.736416,0.746885,1.320480,1.219976,1.072735,...,1.025818,1.367994,1.390146,1.119316,1.278916,0.824775,1.466405,1.244221,0.852335,1.269582
SW219171-1,10.0,SP0135,J03,1.401852,1.330168,1.284220,0.822720,0.623932,1.035767,0.626880,1.307221,1.196283,1.065542,...,1.081208,1.433592,1.277080,0.758471,1.367368,0.634223,1.416111,1.358786,0.760566,1.515628


,,,CompoundName,SW219778-1,SW219366-1,SW218151-2,SW219425-1,SW219079-1,SW222338-1,SW219396-1,SW196607-4,SW219879-1,SW219768-1,...,SW219416-1,SW218112-2,SW219754-1,SW220254-1,SW219869-1,SW219664-1,SW197554-3,SW219415-1,SW199536-4,SW218141-2
,,,Concentration,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
,,,PlateName,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,...,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142,SP0142
,,,WellID,L08,O19,I04,K11,J06,B04,J05,G20,H11,M13,...,P20,C17,D20,K16,L20,I09,J16,J21,M09,E15
CompoundName,Concentration,PlateName,WellID,,,,,,,,,,,,,,,,,,,,,
SW218124-2,10.0,SP0135,E03,-0.125537,0.016927,-0.331452,0.095889,0.165803,0.066495,0.312833,-0.377220,-0.265864,0.253917,...,0.114873,-0.234141,-0.173954,-0.227687,0.312663,0.444563,-0.276599,-0.069406,0.448051,-0.072620
SW219896-1,10.0,SP0135,O15,-0.247795,-0.024736,-0.511265,-0.294668,0.245081,0.314229,0.328884,-0.197581,-0.184858,-0.017520,...,0.062417,-0.131026,-0.254594,-0.295335,0.025144,0.300142,-0.324513,0.029176,0.268430,-0.080931
SW197639-3,10.0,SP0135,N08,0.372787,0.268461,0.354381,-0.153401,-0.253436,-0.080003,-0.257563,0.266507,0.238981,0.132082,...,-0.103231,0.304599,0.276613,-0.120444,0.065215,-0.300075,0.345005,0.104350,-0.224923,0.365985
SW218086-2,10.0,SP0135,G22,-0.356639,-0.220774,-0.513809,-0.122234,0.200482,0.263584,0.253115,-0.320480,-0.219976,-0.072735,...,-0.025818,-0.367994,-0.390146,-0.119316,-0.278916,0.175225,-0.466405,-0.244221,0.147665,-0.269582
SW219171-1,10.0,SP0135,J03,-0.401852,-0.330168,-0.284220,0.177280,0.376068,-0.035767,0.373120,-0.307221,-0.196283,-0.065542,...,-0.081208,-0.433592,-0.277080,0.241529,-0.367368,0.365777,-0.416111,-0.358786,0.239434,-0.515628


### Original function I made

Note: this is very slow

In [3]:
def createXLSheet(distanceReport: pd.DataFrame, pearsonReport: pd.DataFrame, outName: str):
    assert list(distanceReport.columns) == list(pearsonReport.columns)
    
    print("Creating Excel file...", file=sys.stderr)
    with pd.ExcelWriter(outName) as f:
        for i in distanceReport.columns: #this actually takes so long to process, it might be better to cache some files
            
            pageName = "._.".join([str(x) for x in i])
            print(f'...Working on: {pageName}...', file=sys.stderr)

            combDf = pd.concat([pearsonReport[i], distanceReport[i]], axis=1)
            combDf.columns = ['Pearson_R', 'Corr_Dist']

            combDf.to_excel(f, sheet_name=pageName)
    print("...Completed!", file=sys.stderr)
    return

In [4]:
# createXLSheet(distanceReport=dist, pearsonReport=pearson, outName="huh.xlsx")

# takes:
# 2.26 s ± 29.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

## MAKE IT FAST!

In [5]:
from multiprocessing import Pool, Queue, Manager, Process, cpu_count #using multiprocessing

In [6]:
def createSubDf(i, distDf, pearsonDf): # instead of concatinating why not create a new df?
    pageName = "._.".join(map(str, i))
    combDf = pd.DataFrame({
        'Pearson_R': pearsonDf[i],
        'Corr_Dist': distDf[i]
    })
    return combDf, pageName

In [7]:
def createXLSheetOptimised(distanceReport: pd.DataFrame, pearsonReport: pd.DataFrame, outName: str):
    assert distanceReport.columns.equals(pearsonReport.columns)
    
    sheets = []
    print("Creating Excel file...", file=sys.stderr)

    with Pool() as pool:
        processes = [pool.apply_async(createSubDf, args=(i, distanceReport, pearsonReport)) for i in distanceReport.columns]

        for process in processes:
            sheets.append(process.get())
    
    with pd.ExcelWriter(outName) as f:
        for df, page in sheets:
            print(f'...Working on: {page}...', file=sys.stderr)
            
            df.to_excel(f, sheet_name=page)

In [8]:
# createXLSheetOptimised(distanceReport=dist, pearsonReport=pearson, outName="huh.xlsx")
#this took 9m 27.4s
#crazy that this is slower (hardware dependent?)

### Trying the Queue and Worker method

In [9]:
def writerWorker(q: Queue, outName: str):
    with pd.ExcelWriter(outName, mode='w') as f:
        while True:
            data = q.get()
            if data is None:
                #f.save()
                break
            print(f'...Working on: {data[1]}...', file=sys.stderr)
            data[0].to_excel(f, sheet_name=data[1])

In [10]:
def createXLSheetOptimisedVersion2(distanceReport: pd.DataFrame, pearsonReport: pd.DataFrame, outName: str):
    assert distanceReport.columns.equals(pearsonReport.columns)
    
    sheets = []
    print("Creating Excel file...", file=sys.stderr)

    with Pool() as pool, Manager() as manager:
        q = manager.Queue()
        
        #writer process
        writerProcess = Process(target=writerWorker, args=(q, outName))
        writerProcess.start()

        processes = [pool.apply_async(createSubDf, args=(i, distanceReport, pearsonReport)) for i in distanceReport.columns]

        for process in processes:
            sheets.append(process.get())
        
        for sheet in sheets:
            q.put(sheet)
        
        q.put(None)
        writerProcess.join()
        
    print("...Completed!", file=sys.stderr)

In [11]:
# createXLSheetOptimisedVersion2(distanceReport=dist, pearsonReport=pearson, outName="huh.xlsx")
# this does not work sadly

#### let's try a different method

In [12]:
def createXLSheetOptimisedVersion3(distanceReport: pd.DataFrame, pearsonReport: pd.DataFrame, outName: str):
    assert distanceReport.columns.equals(pearsonReport.columns)
    
    sheets = []
    print("Creating Excel file...", file=sys.stderr)

    writerProcess = []
    with Pool() as pool:
        processes = [pool.apply_async(createSubDf, args=(i, distanceReport, pearsonReport)) for i in distanceReport.columns]

        for process in processes:
            sheets.append(process.get())
        
    num_cpus = cpu_count()
    print(f'CPU count: {num_cpus}', file=sys.stderr)

    q = Queue()

    for _ in range(num_cpus):
        process = Process(target=writerWorker, args=(q, outName))
        writerProcess.append(process)
        process.start()

    for sheet in sheets:
        q.put(sheet)
    
    for _ in range(num_cpus):
        q.put(None)
    
    for process in writerProcess:
        process.join()
        
    print("...Completed!", file=sys.stderr)

In [13]:
# createXLSheetOptimisedVersion3(distanceReport=dist, pearsonReport=pearson, outName="huh.xlsx")
#Thats damn fast but outputs are not generated (50s-1min)

#### Let's try this

In [14]:
def write_chunk_to_excel(chunk, out_name):
    with pd.ExcelWriter(out_name, engine='openpyxl', mode='a') as writer:
        for sheet_name, df in chunk.items():
            print(f'...Working on: {data[1]}...', file=sys.stderr)
            df.to_excel(writer, sheet_name=sheet_name, index=True)


In [15]:
def createXLSheetOptimisedVersion4(distanceReport: pd.DataFrame, pearsonReport: pd.DataFrame, outName: str):
    assert distanceReport.columns.equals(pearsonReport.columns)
    
    sheets = []
    print("Creating Excel file...", file=sys.stderr)

    num_cpus = cpu_count()
    print(f'CPU count: {num_cpus}', file=sys.stderr)

    chunk_size = len(distanceReport.columns) // num_cpus

    sheets = [createSubDf(i, distanceReport, pearsonReport) for i in distanceReport.columns]
    
    # with Pool(processes=num_cpus) as pool:
    #     processes = [pool.apply_async(createSubDf, args=(i, distanceReport, pearsonReport)) for i in distanceReport.columns]

    #     for process in processes:
    #         sheets.append(process.get())
    # writer = pd.ExcelWriter(outName, engine='openpyxl', mode='w')
    #writer.close()
    processes=[]
    with Pool(processes=num_cpus) as pool:
        for start in range(0, len(sheets), chunk_size):
            chunk = {sheet[1]: sheet[0] for sheet in sheets[start:start+chunk_size]}
            process = pool.apply_async(write_chunk_to_excel, args=(chunk, outName))
            processes.append(process)
        
        for process in processes:
            process.get()
        
    print("...Completed!", file=sys.stderr)

In [16]:
# createXLSheetOptimisedVersion4(distanceReport=dist, pearsonReport=pearson, outName="huh.xlsx")

#### I am goin to attempt to divide up the data and then parallelize the process

In [17]:
def chunkToExcel(chunk, outName):
    with pd.ExcelWriter(outName, engine='openpyxl') as writer:
        for sheet_name, df in chunk.items():
            print(f'...Working on: {sheet_name}...', file=sys.stderr)
            df.to_excel(writer, sheet_name=sheet_name, index=True)

In [18]:
def processHandler(startIndex, endIndex, data: list, notebookDir):
    chunk = {d[1]: d[0] for d in data[startIndex:endIndex]}
    nbName = f'temp_{startIndex}.xlsx'
    name = os.path.join(os.getcwd(), '.temp/', nbName)
    
    chunkToExcel(chunk=chunk, outName=name)

In [19]:
def parallelWrite(distanceReport: pd.DataFrame, pearsonReport: pd.DataFrame):
    cwd = os.getcwd()

    try:
        os.mkdir('.temp/')
    except:
        os.rmdir('.temp/')
        os.mkdir('.temp/')

    sheets = [createSubDf(i, distDf=distanceReport, pearsonDf=pearsonReport) for i in distanceReport.columns]
    numCPUs = cpu_count() - 2 # be very careful with this
    chunkSize = len(sheets) // numCPUs
    print(f'Number of CPUs used (not total number of CPUs): {numCPUs}', file=sys.stderr)
    print(f'Chunk Size: {chunkSize}', file=sys.stderr)

    processes = []
    for start in range(0, len(sheets), chunkSize):
        end = start + chunkSize if start + chunkSize <= len(sheets) else len(sheets)
        # processHandler(start, end, sheets, cwd)
        process = Process(target=processHandler, args=(start, end, sheets, cwd))
        processes.append(process)
        process.start()

    for process in processes:
        process.join()

In [20]:
os.chdir('/home/derfelt/featReport/testInputs/')
print(os.getcwd())
os.mkdir('.temp/')
os.rmdir('.temp/')

cwd = os.getcwd()
nbName = f'temp.xlsx'
os.path.join(cwd,'.temp/',nbName)

/home/derfelt/featReport/testInputs


'/home/derfelt/featReport/testInputs/.temp/temp.xlsx'

In [21]:
# parallelWrite(distanceReport=dist, pearsonReport=pearson)

# IT WORKS WITH A 3 MIN RUNTIME

# Takeaways -> do not use all CPUs on machine, instead use like 1 or 2 minus the number of CPUs 

#### Now let's combine all the files into one xlsx sheet

In [22]:
from openpyxl import  load_workbook, Workbook

path = os.path.join(os.getcwd(), '.temp/')

tempNotebooks = []
if os.path.exists(path=path):
    tempNotebooks = [os.path.join(path, file) for file in os.listdir(path)] #these are all notebook paths
else:
    print(".temp/ not found :(", file=sys.stderr)
    exit(-1)


.temp/ not found :(


In [23]:
def combNotebooks(nbPaths, mergedFile): # I may want to look at options to multt-thread this in the future
    mergedWb = Workbook()

    for nbPath in nbPaths:
        workbook = load_workbook(filename=nbPath, read_only=True)

        for sheet in workbook.sheetnames:
            ws = workbook[sheet]
            newWs = mergedWb.create_sheet(title = sheet)

            for row in ws.iter_rows():
                newRow = [cell.value for cell in row]
                newWs.append(newRow)

    mergedWb.save(mergedFile)

In [24]:
combNotebooks(tempNotebooks, mergedFile='huh.xlsx')
# not as bad as I though (def. faster)

In [25]:
# remove the files
def cleanTempFiles(folderPath):
    if os.path.exists(folderPath):
        for file in os.listdir(folderPath):
            filePath = os.path.join(folderPath, file)

            if os.path.isfile(filePath):
                os.remove(filePath)
        os.rmdir(folderPath)
    return

cleanTempFiles(path)

#### Now, let's combine all the things we have discovered into one

In [27]:
from time import sleep
# create temp files
parallelWrite(distanceReport=dist, pearsonReport=pearson)
sleep(1) # need to wait for files to flush to drive

#find temp files
path = os.path.join(os.getcwd(), '.temp/')

tempNotebooks = []
if os.path.exists(path=path):
    tempNotebooks = [os.path.join(path, file) for file in os.listdir(path)] #these are all notebook paths
else:
    print(".temp/ not found :(", file=sys.stderr)
    exit(-1)

#merge notebooks
combNotebooks(tempNotebooks, mergedFile='out.xlsx')
cleanTempFiles(path)


Number of CPUs used (not total number of CPUs): 10
Chunk Size: 32
...Working on: SW219778-1._.2.0._.SP0142._.L08...
...Working on: SW219302-1._.2.0._.SP0142._.K06...
...Working on: SW220258-1._.2.0._.SP0142._.P14...

...Working on: SW202555-3._.2.0._.SP0142._.I13......Working on: SW220132-1._.2.0._.SP0142._.P05...
...Working on: SW218101-2._.2.0._.SP0142._.I07...
...Working on: SW219463-1._.2.0._.SP0142._.L21...
...Working on: SW218254-2._.2.0._.SP0142._.I14...
...Working on: SW219803-1._.2.0._.SP0142._.G16...
...Working on: SW219398-1._.2.0._.SP0142._.A10...
...Working on: SW196972-3._.2.0._.SP0142._.F13...
...Working on: SW219366-1._.2.0._.SP0142._.O19...
...Working on: SW197637-2._.2.0._.SP0142._.E16...
...Working on: SW219171-1._.2.0._.SP0142._.J03...
...Working on: SW218076-2._.2.0._.SP0142._.A20......Working on: SW219256-1._.2.0._.SP0142._.H03...

...Working on: SW196369-4._.2.0._.SP0142._.H19...
...Working on: SW220010-1._.2.0._.SP0142._.L15......Working on: SW222227-1._.2.0._.S